In [1]:
# 다양한 방식으로 MariaDB 테이블 읽고 스기
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (
SparkConf()
    .setAppName("MY_MariaDB") # replace with your desired name
    .set("spark.jars.packages", \
         "org.mariadb.jdbc:mariadb-java-client:3.3.3"\
        )
    # .set("spark.sql.shuffle.partitions", "4") # default is 200 partitions which is too many for local
    # .setMaster("local[*]") # replace the * with your desired number of cores. * for use all.
    .setMaster("spark://34.125.136.103:30077")
)

spark = SparkSession.builder \
    .config(conf=conf)\
    .getOrCreate()

:: loading settings :: url = jar:file:/root/miniconda3/envs/spark341/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.mariadb.jdbc#mariadb-java-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-53930681-ac63-4cf6-89e1-0d37e3be6f37;1.0
	confs: [default]
	found org.mariadb.jdbc#mariadb-java-client;3.3.3 in central
	found com.github.waffle#waffle-jna;3.3.0 in central
	found net.java.dev.jna#jna;5.13.0 in central
	found net.java.dev.jna#jna-platform;5.13.0 in central
	found org.slf4j#jcl-over-slf4j;2.0.7 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.3 in central
	found com.google.errorprone#error_prone_annotations;2.10.0 in central
	found org.checkerframework#checker-qual;3.32.0 in central
:: resolution report :: resolve 815ms :: artifacts dl 27ms
	:: modules in use:
	com.github.ben-manes.caffeine#caffeine;2.9.3 from central in [default]
	com.github.waffle#waffle-jna;3.3.0 from central in [default]
	com

In [2]:
# 접속 정보
jdbcHostname = "34.125.136.103"
jdbcDatabase = "sparkdb"
jdbcPort = 30007
jdbcUrl = "jdbc:mysql://{0}:{1}/{2}?permitMysqlScheme".format(jdbcHostname, jdbcPort, jdbcDatabase)
jdbcTable = "test"
properties = {
  "driver" : "org.mariadb.jdbc.Driver",
  "user"   : "root",
  "password" : "tjehdgml"
}

In [4]:
# 데이터 조회
# query option은 read에서만 사용 가
sql = "\
    SELECT *\
    FROM   test\
"
df = spark.read.format("jdbc") \
                    .option("url", jdbcUrl) \
                    .option("driver", properties["driver"]) \
                    .option("query", sql) \
                    .option("user", properties["user"]) \
                    .option("password", properties["password"]) \
                    .load()
                    
df.show(5)

+---+-------+
| id|   name|
+---+-------+
|  1|example|
+---+-------+



In [7]:
# MariaDB에 새 테이블 생성 및 데이터 삽입
# sql = "
#     CREATE TABLE IF NOT EXISTS test (\
#         id INT AUTO_INCREMENT PRIMARY KEY,\
#         name VARCHAR(255)\
#     )\
# "

# 데이터 삽입
columns = ["name"]
data = [("James"),("Ann"),("Jeff"),("Jennifer"),("Seoddong"),("Eiloy"),("Eve")]

sampleDF = spark.sparkContext.parallelize(data).toDF(columns)


sampleDF.write.format("jdbc") \
                .option("url", jdbcUrl) \
                .option("driver", properties["driver"]) \
                .option("dbtable", jdbcTable) \
                .option("user", properties["user"]) \
                .option("password", properties["password"]) \
                .save()

TypeError: Can not infer schema for type: <class 'str'>

In [8]:
# 스파크 세션 종료
spark.stop()